* 인프런 강의의 수업자료 입니다
* 인프런 강의 주소 : https://inf.run/FX4TP



## Hugging Face의 transformers 라이브러리를 사용하여 한국어 GPT-2 모델을 활용하여 문장 생성하기

In [1]:
# transformers는 Hugging Face에서 제공하는 자연어처리 라이브러리로, 다양한 사전학습 언어모델을 제공하고 있습니다.
# accelerate는 Hugging Face에서 제공하는 학습 가속 라이브러리로, PyTorch와 함께 사용할 수 있습니다.
# 이를 이용하면 단일 노드의 다중 GPU 학습을 지원하고, 학습시간을 단축할 수 있습니다.
!pip install transformers --upgrade --q
!pip install accelerate --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.6 MB/s eta 0:00:00


## koGPT2
* [skt/kogpt2-base-v2 · Hugging Face](https://huggingface.co/skt/kogpt2-base-v2)

* tokenizers 패키지의 Character BPE tokenizer로 학습되었습니다.


* 사전 크기는 51,200 이며 대화에 자주 쓰이는 아래와 같은 이모티콘, 이모지 등을 추가하여 해당 토큰의 인식 능력을 올렸습니다.
* 또한 <unused0> ~ <unused99>등의 미사용 토큰을 정의해 필요한 테스크에 따라 자유롭게 정의해 사용할 수 있게 했습니다.

* BPE(Byte Pair Encoding)은 하나의 텍스트를 토큰으로 분리하기 위한 subword 기반 알고리즘이며, 유사한 알고리즘으로는 WordPiece, SentencePiece 등이 있습니다. 이들 알고리즘은 모두 텍스트를 일정 크기의 subword 단위로 쪼개서 처리하는 과정에서 OOV(Out-Of-Vocabulary) 문제를 해결하고자 개발된 알고리즘입니다.
    * Wordpiece 모델에서 Out-of-vocabulary(OOV) 문제를 해결하기 위해, 모든 단어를 서브워드(subword) 단위로 분할하여 처리합니다. 즉, 모든 단어를 이루는 subword들의 조합으로 단어를 대체합니다. 예를 들어 "playing"이라는 단어를 Wordpiece 모델에 넣으면 "play"와 "##ing"으로 분할하여 처리합니다. "##"은 서브워드를 나타내는 특별한 기호입니다. 이렇게 단어를 subword들의 집합으로 분할하여 처리함으로써, 이전에 OOV 문제로 처리할 수 없었던 단어를 분할하여 처리할 수 있게 되어 모델의 성능을 향상시킬 수 있습니다.
    * SentencePiece는 Google에서 개발한 오픈소스 기반의 자연어 처리 도구입니다. BPE(Byte Pair Encoding)와 Unigram Language Model Tokenizer(ULM)를 적용할 수 있으며, 모델의 입력으로 사용할 수 있는 단어 집합을 만들기 위해 사용됩니다.
    * SentencePiece는 형태소 분석 및 품사 태깅을 수행하지 않고, 문자열을 작은 서브워드(subword)로 분할합니다. 이를 통해, OOV(Out of Vocabulary) 문제를 완화하고, 모델의 성능을 높일 수 있습니다. 또한, SentencePiece는 대규모 텍스트 데이터에서 자동으로 언어 모델을 학습하므로, 사전에 훈련된 모델을 사용하여 적은 데이터로도 효과적인 자연어 처리를 수행할 수 있습니다.
    * SentencePiece는 PyTorch, TensorFlow, C++, Java, Python 등에서 사용할 수 있습니다. 또한, 다양한 언어(한국어, 영어, 일본어 등)를 지원하며, 사용자가 직접 모델을 학습할 수 있도록 API를 제공합니다.



## PreTrainedTokenizerFast

* tokenizer를 생성합니다. from_pretrained 함수를 사용하여 사전 학습된 한국어 GPT-2 모델을 불러옵니다. 이때, 시작 토큰, 종료 토큰, 알 수 없는 토큰, 패딩 토큰, 마스크 토큰 등을 설정할 수 있습니다.

* 마스크 토큰은 자연어 처리에서 문장 내 일부 단어를 마스킹하여 모델이 이를 예측하는 작업입니다. 마스킹 작업은 모델이 문장 내에서 해당 단어가 빠진 문맥을 이해하고, 문맥에 맞는 적절한 단어를 예측하는 능력을 강화시키는 데에 유용합니다.

* 마스크 토큰은 대부분의 transformer 모델에서 사용되며, BERT, RoBERTa, GPT 등에서 사용됩니다. 대표적으로 BERT에서는 15%의 단어를 마스킹하며, 이 중 80%는 [MASK] 토큰으로, 10%는 랜덤한 단어로, 10%는 원래 단어로 대체됩니다.

* 한국어 GPT-2 모델에서도 마스크 토큰을 지원하며, tokenizer에서 mask_token 옵션을 사용하여 마스크 토큰을 설정할 수 있습니다. 마스크 토큰을 사용하는 것은 모델의 학습 능력을 향상시키는 데에 큰 도움을 줄 수 있습니다.


* bos_token : 토크나이저에서 사용되는 특수 토큰 중 하나로, 문장의 시작을 나타내는 토큰입니다. BOS는 "Beginning of Sentence"의 약자로, 이 토큰이 삽입된 위치가 문장의 시작임을 나타냅니다. BOS 토큰은 토크나이징된 시퀀스의 맨 앞에 추가됩니다. 예를 들어, "Hello, world!"라는 문장을 BERT 토크나이저로 토큰화하면 "[CLS] Hello, world! [SEP]"와 같이 BOS 토큰이 "[CLS]"로 추가되어 있습니다.
* eos_token: 문자열의 끝을 나타내는 토큰입니다.
* unk_token: 알 수 없는 토큰입니다. 모델이 학습하지 않은 단어를 나타낼 때 사용됩니다.
* pad_token: 시퀀스 패딩에 사용되는 토큰입니다. 시퀀스의 길이를 일정하게 맞추기 위해 사용됩니다.
* mask_token: 마스킹 토큰입니다. 모델의 입력 시퀀스 중 일부 토큰을 마스킹하여 학습에 사용됩니다.

* [Tokenizer](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/tokenizer#transformers.PreTrainedTokenizerFast)
* [skt/kogpt2-base-v2 · Hugging Face](https://huggingface.co/skt/kogpt2-base-v2)

In [3]:
# PreTrainedTokenizerFast
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>',
                                                    eos_token='</d>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')
len(tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


51200

In [6]:
raw_text = "안녕하세요. 한국어 GPT-2 입니다.😤 감사합니다."
raw_text

'안녕하세요. 한국어 GPT-2 입니다.😤 감사합니다.'

In [8]:
# tokenize(): 문장을 token으로 나눠서 반환
tokens = tokenizer.tokenize(raw_text)
print(tokens)

['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', '▁감사', '합니다.']


In [10]:
# convert_tokens_to_ids(): token 목록을 받아서 token_id 목록을 반환
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[25906, 8702, 7801, 25856, 34407, 10528, 422, 426, 18258, 9241, 12521, 254, 15940, 37194]


In [13]:
# convert_ids_to_tokens(): token_id 목록을 받아서 token 목록을 반환
converted_tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(converted_tokens)

['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', '▁감사', '합니다.']


In [14]:
# convert_tokens_to_string(): token 목록을 받아서 문장으로 반환
converted_text = tokenizer.convert_tokens_to_string(converted_tokens)
converted_text

'안녕하세요. 한국어 GPT-2 입니다.😤 감사합니다.'

In [17]:
# encode(): token으로 자르기 + token을 token_id로 변환하여 반환
# = tokenize() + convert_tokens_to_ids()
tokenizer.encode(raw_text)

[25906,
 8702,
 7801,
 25856,
 34407,
 10528,
 422,
 426,
 18258,
 9241,
 12521,
 254,
 15940,
 37194]

In [18]:
# tokenizer가 갖고 있는 특별한 token들을 확인
tokenizer.bos_token, tokenizer.eos_token, tokenizer.unk_token, tokenizer.pad_token, tokenizer.mask_token

('</s>', '</d>', '<unk>', '<pad>', '<mask>')

In [19]:
# tokenizer가 갖고 있는 특별한 token들의 token_id를 확인
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.unk_token_id, tokenizer.pad_token_id, tokenizer.mask_token_id

(1, 8, 5, 3, 6)

In [20]:
# UNK: 학습하지 못한 입력을 대체
tokenizer.tokenize("뷁")

['▁', '<unk>']

## 다른 문장 토큰화 적용

In [28]:
sample_text = """
    마스킹은 모델 학습에서 매우 유용한 기술 중 하나입니다.
    이를 통해 모델은 입력 시퀀스의 일부를 무작위로 마스킹하여 누락된 토큰을 예측하도록 학습할 수 있습니다.
    이렇게 하면 모델은 입력 시퀀스에서 누락된 정보를 추론하는 능력을 향상시키며,
    더 일반화된 학습이 가능해집니다.
    예를 들어, 기계 번역에서는 마스킹을 사용하여 모델이 문장의 일부 단어를 예측하도록 학습시킬 수 있습니다.
    이를 통해 모델은 문장의 구조와 문맥을 이해하고, 번역의 정확도를 향상시킬 수 있습니다.
    또한, 마스킹은 모델이 오버피팅(overfitting)을 피하는 데도 도움이 됩니다.
    오버피팅은 모델이 학습 데이터에 지나치게 적합하여 새로운 데이터에 대한 일반화 능력이 부족한 상태를 의미합니다.
    마스킹을 통해 모델은 입력 시퀀스의 랜덤한 부분을 누락시킴으로써 일반화 능력을 향상시키고, 오버피팅을 방지할 수 있습니다.
    따라서, 마스킹은 모델 학습에서 중요한 역할을 합니다. 이를 통해 모델은 누락된 정보를 추론하는 능력을 강화하고,
    오버피팅을 방지하며, 더 일반화된 학습이 가능해집니다.
    """
sample_text

'\n    마스킹은 모델 학습에서 매우 유용한 기술 중 하나입니다.\n    이를 통해 모델은 입력 시퀀스의 일부를 무작위로 마스킹하여 누락된 토큰을 예측하도록 학습할 수 있습니다.\n    이렇게 하면 모델은 입력 시퀀스에서 누락된 정보를 추론하는 능력을 향상시키며,\n    더 일반화된 학습이 가능해집니다.\n    예를 들어, 기계 번역에서는 마스킹을 사용하여 모델이 문장의 일부 단어를 예측하도록 학습시킬 수 있습니다.\n    이를 통해 모델은 문장의 구조와 문맥을 이해하고, 번역의 정확도를 향상시킬 수 있습니다.\n    또한, 마스킹은 모델이 오버피팅(overfitting)을 피하는 데도 도움이 됩니다.\n    오버피팅은 모델이 학습 데이터에 지나치게 적합하여 새로운 데이터에 대한 일반화 능력이 부족한 상태를 의미합니다.\n    마스킹을 통해 모델은 입력 시퀀스의 랜덤한 부분을 누락시킴으로써 일반화 능력을 향상시키고, 오버피팅을 방지할 수 있습니다.\n    따라서, 마스킹은 모델 학습에서 중요한 역할을 합니다. 이를 통해 모델은 누락된 정보를 추론하는 능력을 강화하고,\n    오버피팅을 방지하며, 더 일반화된 학습이 가능해집니다.\n    '

In [24]:
# tokenizer.tokenize
tokenizer_text = tokenizer.tokenize(sample_text)
print(tokenizer_text)

['▁\n', '▁', '▁', '▁', '▁마', '스', '킹', '은', '▁모델', '▁학습', '에서', '▁매우', '▁유용한', '▁기술', '▁중', '▁하나', '입니다.\n', '▁', '▁', '▁', '▁이를', '▁통해', '▁모델', '은', '▁입력', '▁시', '퀀', '스의', '▁일부를', '▁무작', '위로', '▁마', '스', '킹', '하여', '▁누락', '된', '▁토', '큰', '을', '▁예측', '하도록', '▁학습', '할', '▁수', '▁있습니다.\n', '▁', '▁', '▁', '▁이렇게', '▁하면', '▁모델', '은', '▁입력', '▁시', '퀀', '스에서', '▁누락', '된', '▁정보를', '▁추론', '하는', '▁능력을', '▁향상', '시키며,', '\n', '▁', '▁', '▁', '▁더', '▁일반', '화된', '▁학습', '이', '▁가능', '해', '집니다.\n', '▁', '▁', '▁', '▁예를', '▁들어,', '▁기계', '▁번역', '에서는', '▁마', '스', '킹', '을', '▁사용하여', '▁모델이', '▁문장의', '▁일부', '▁단어를', '▁예측', '하도록', '▁학습', '시킬', '▁수', '▁있습니다.\n', '▁', '▁', '▁', '▁이를', '▁통해', '▁모델', '은', '▁문장의', '▁구조와', '▁문', '맥을', '▁이해', '하고,', '▁번역', '의', '▁정확', '도를', '▁향상', '시킬', '▁수', '▁있습니다.\n', '▁', '▁', '▁', '▁또한,', '▁마', '스', '킹', '은', '▁모델이', '▁오버', '피', '팅', '(', 'over', 'f', 'itt', 'ing', ')을', '▁피하는', '▁데도', '▁도움이', '▁됩니다.\n', '▁', '▁', '▁', '▁오버', '피', '팅', '은', '▁모델이', '▁학습', '▁데이터', '에', '▁지나치게', '▁적

In [ ]:
# tokenizer.encode
tokenizer.encode(sample_text)

In [26]:
tokenizer.convert_tokens_to_string(tokenizer_text)

'\n    마스킹은 모델 학습에서 매우 유용한 기술 중 하나입니다.\n    이를 통해 모델은 입력 시퀀스의 일부를 무작위로 마스킹하여 누락된 토큰을 예측하도록 학습할 수 있습니다.\n    이렇게 하면 모델은 입력 시퀀스에서 누락된 정보를 추론하는 능력을 향상시키며,\n    더 일반화된 학습이 가능해집니다.\n    예를 들어, 기계 번역에서는 마스킹을 사용하여 모델이 문장의 일부 단어를 예측하도록 학습시킬 수 있습니다.\n    이를 통해 모델은 문장의 구조와 문맥을 이해하고, 번역의 정확도를 향상시킬 수 있습니다.\n    또한, 마스킹은 모델이 오버피팅(overfitting)을 피하는 데도 도움이 됩니다.\n    오버피팅은 모델이 학습 데이터에 지나치게 적합하여 새로운 데이터에 대한 일반화 능력이 부족한 상태를 의미합니다.\n    마스킹을 통해 모델은 입력 시퀀스의 랜덤한 부분을 누락시킴으로써 일반화 능력을 향상시키고, 오버피팅을 방지할 수 있습니다.\n    따라서, 마스킹은 모델 학습에서 중요한 역할을 합니다. 이를 통해 모델은 누락된 정보를 추론하는 능력을 강화하고,\n    오버피팅을 방지하며, 더 일반화된 학습이 가능해집니다.\n    '

## GPT2LMHeadModel

 GPT2LMHeadModel 클래스를 사용하여 모델을 불러옵니다. from_pretrained 함수를 사용하여 사전 학습된 한국어 GPT-2 모델을 불러옵니다.

GPT2LMHeadModel는 OpenAI에서 개발한 언어 모델인 GPT-2(GPT-2 Language Model)을 파이토치(PyTorch)로 구현한 모델입니다.

GPT-2는 Transformer 아키텍처를 기반으로 한 언어 모델로, 입력된 문장에 대해 다음 단어를 예측하는 과정에서 이전 단어들의 상호작용을 반영합니다. Transformer는 주로 자연어처리 분야에서 사용되는 딥러닝 모델 중 하나로, 이전의 RNN(Recurrent Neural Network) 모델에 비해 계산 속도가 빠르며, 병렬 처리가 용이하여 학습 시간이 단축됩니다.

GPT2LMHeadModel은 Transformer 모델의 블록인 GPT2Block을 12개 연결한 transformer와, 언어 모델의 출력층인 lm_head로 구성됩니다. transformer는 입력된 텍스트 시퀀스를 이용해 텍스트를 임베딩하고, 자기어텐션(self-attention)과 전방향(feedforward) 신경망으로 구성된 블록인 GPT2Block을 통해 시퀀스의 정보를 추출합니다. lm_head는 transformer의 출력을 입력받아 다음 단어를 예측하는 과정에서 사용됩니다.

따라서 GPT2LMHeadModel는 입력된 텍스트를 이용해 다음 단어를 예측하는 언어 모델입니다. 이를 통해 대화 시스템, 번역, 요약 등의 다양한 자연어 처리 태스크를 수행할 수 있습니다.

* [skt/kogpt2-base-v2 · Hugging Face](https://huggingface.co/skt/kogpt2-base-v2)
* [OpenAI GPT2](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/gpt2#transformers.GPT2LMHeadModel)

In [29]:
# GPT2LMHeadModel.from_pretrained
# skt/kogpt2-base-v2
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을



### GPT2Model 사용하기

* GPT2LMHeadModel은 GPT-2 모델의 언어 모델링 버전으로, 입력 시퀀스에 대해 다음 단어를 예측하는 작업을 수행합니다.
* 이 모델은 GPT2Model과 LM Head로 구성됩니다.
* GPT2Model은 GPT-2 아키텍처를 구현한 모듈이며, 입력 시퀀스를 받아 transformer 레이어를 거치며 출력값을 내보냅니다. 이 모델은 다음과 같은 하위 모듈들로 구성됩니다.
* WTE(Word Token Embedding)는 각 단어를 고정 크기의 벡터로 변환하는 기술로, 단어를 벡터 공간에 임베딩하여 자연어 처리 모델에 입력으로 제공합니다. 이를 통해 단어 간 유사도를 측정하거나, 텍스트 데이터의 차원을 줄이는 등 다양한 자연어 처리 작업에서 사용됩니다. WTE는 일반적으로 원-핫 인코딩 방법과 달리, 단어의 의미를 나타내는 잠재적인 구조를 학습하도록 설계됩니다. 이를 위해 WTE는 주로 언어 모델에 사용되는 신경망 아키텍처인 임베딩 레이어와 결합하여 사용됩니다.

    * wte: 입력 토큰의 임베딩 테이블입니다.
    * wpe: 입력 토큰의 위치 임베딩 테이블입니다.
    * drop: 입력 토큰의 임베딩과 위치 임베딩에 대한 드롭아웃을 수행합니다.
    * h: GPT-2 블록들의 스택으로 구성된 transformer 레이어입니다.
    * ln_f: 출력값에 대한 Layer Normalization을 수행합니다.


트랜스포머 모델에서 Layer Normalization은 다음과 같은 수식으로 정의됩니다.

$$\text{LayerNorm}(x_i) = \gamma \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$$


트랜스포머 모델에서 Layer Normalization은 각 레이어의 입력을 정규화하는 기법 중 하나입니다. 각 레이어의 입력은 해당 레이어에서 연산이 수행되기 전에 정규화됩니다. 이러한 정규화는 미니배치 내에서 각 샘플의 평균과 분산을 사용하여 수행됩니다. 즉, 레이어별로 입력에 대한 평균과 분산을 구하고, 이를 사용하여 입력을 정규화합니다.

Layer Normalization은 Batch Normalization과 비슷한 목적을 가지지만, Batch Normalization은 미니배치 단위로 입력을 정규화하는 반면, Layer Normalization은 레이어 내에서 입력을 정규화합니다. 이러한 차이점으로 인해 Layer Normalization은 Batch Normalization과 비교해 작은 미니배치나 긴 시퀀스의 경우에도 안정적으로 학습할 수 있는 장점이 있습니다.

Layer Normalization은 RNN과 같이 시퀀스 모델링에서 성능 향상에 효과적인 것으로 알려져 있습니다. 이는 시퀀스 모델링에서 배치 정규화를 적용하기 어려운 점과 관련이 있습니다. 배치 정규화는 미니배치 크기에 따라 성능이 크게 달라질 수 있는 반면, Layer Normalization은 샘플 단위로 계산하기 때문에 미니배치 크기에 영향을 받지 않습니다. 또한 Layer Normalization은 모델의 불안정성을 줄이고 학습 속도를 빠르게 하는 효과도 있습니다.


여기서 $x$는 입력 텐서, $\mu$와 $\sigma^2$는 평균과 분산으로, 미니배치 내에서 샘플별로 계산됩니다. $\gamma$와 $\beta$는 학습 가능한 스케일(scale)과 시프트(shift) 매개변수입니다. $\epsilon$은 작은 값으로 분모가 0이 되는 것을 방지합니다.



* $x_i$ : $i$번째 입력 특성 값
* $\mu$ : 평균
* $\sigma$ : 표준편차
* $\epsilon$ : 작은 값으로 분모가 0이 되는 것을 방지하기 위한 값
* $\gamma$ : 스케일 조정 파라미터
* $\beta$ : 이동 조정 파라미터
* $\text{LN}(x_i)$ : 정규화된 $i$번째 입력 특성 값




### GPT-2 블록

은 self-attention과 MLP 레이어로 구성되어 있습니다. 이 중 self-attention은 입력 시퀀스 내에서 다양한 위치들 간의 상호작용을 수행하여, 특정 위치의 출력값이 전체 시퀀스의 컨텍스트에 따라 동적으로 결정되는 효과를 가집니다.

#### LM Head

LM Head는 transformer 레이어의 출력값을 받아, 다음 단어 예측을 수행하는 선형 레이어입니다. 이 레이어는 단어의 분포를 출력값으로 내보내며, 이 분포에서 가장 높은 확률을 가지는 단어가 모델의 예측값이 됩니다.


* https://jalammar.github.io/illustrated-transformer/

<img src="https://nlpinkorean.github.io/images/transformer/transformer_resideual_layer_norm_3.png">

In [30]:
# model
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

* 참고 :
* Conv1D와 Dot Product 모두 벡터 간의 곱셈을 수행합니다. 그러나 Conv1D는 한 쪽 벡터를 이동시키면서 다른 쪽 벡터와의 내적을 계산하는 반면, Dot Product는 두 벡터의 요소 간 곱의 합으로 구성됩니다.

* Conv1D는 커널(kernel)을 이동시키면서 입력 벡터와의 내적을 계산합니다. 이 때 커널의 크기는 고정되어 있지만, 이동하는 위치에 따라 결과가 달라질 수 있습니다. 이를 통해 입력 벡터 내의 패턴을 인식하거나 특징(feature)을 추출하는 등의 역할을 수행할 수 있습니다.

* 반면, Dot Product는 두 벡터의 요소 간 곱의 합으로 구성됩니다. 이를 통해 두 벡터 간의 유사도를 측정하거나, 각 요소가 독립적인 경우에 대해 연산을 수행할 수 있습니다.

* 따라서 Conv1D와 Dot Product는 비슷한 연산이지만, Conv1D는 입력 벡터 내에서 패턴을 인식하는 데에 적합하고, Dot Product는 벡터 간의 유사도 측정 등에 적합합니다.

## 생성
generate 함수를 사용하여 모델을 이용하여 문장을 생성합니다. input_ids에 입력으로 넣을 텍스트를 tokenize한 결과를 Tensor 형태로 입력합니다. max_length는 최대 생성 문장 길이, repetition_penalty는 반복되는 단어를 억제하는 페널티 값입니다. pad_token_id, eos_token_id, bos_token_id는 tokenizer에서 설정한 값과 동일하게 입력해주어야 합니다.

마지막으로, decode 함수를 사용하여 생성된 Tensor 값을 텍스트로 디코딩합니다.

* tokenizer.encode(text, return_tensors='pt'): 주어진 text를 tokenizer를 활용하여 토큰화하고, 이를 pytorch tensor 형태로 변환합니다.
* model.generate(): pytorch tensor 형태로 변환된 input_ids를 GPT2 모델에 입력으로 넣어 자연어 생성을 수행합니다. max_length는 생성할 문장의 최대 길이를 의미하며, repetition_penalty는 반복을 방지하는데 사용되는 계수입니다. pad_token_id, eos_token_id, bos_token_id는 각각 패딩, 문장 끝, 문장 시작을 의미하는 토큰의 id를 나타냅니다. use_cache는 이전 단계의 계산 결과를 재사용할지 여부를 나타냅니다.
* tokenizer.decode(gen_ids[0]): GPT2 모델의 출력값인 gen_ids를 tokenizer를 활용하여 토큰을 텍스트로 디코딩합니다.


In [31]:
text = '꾸준한 운동은 '

input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

꾸준한 운동은 괜찮지만 운동량이 부족하면 오히려 관절에 무리를 줄 수 있다.
관절염으로 인한 통증은 주로 무릎이나 허리, 엉덩이 등 관절이 많이 아픈 부위에 나타난다.
이런 경우라면 평소 운동을 꾸준히 하는 것이 좋다.
특히, 걷기 운동이 가장 좋은데, 특히나 계단 오르내리기, 자전거 타기와 같은 유산소운동을 할 때 통증이 심하다면 전문의를 찾아 정확한 진단을 받는 게 중요하다.
또한 등산 시에도 무리하지 말고 가벼운 스트레칭을 통해 근육과 인대를 강화시키는 것도 도움이 된다.
운동 후 바로 샤워를 하거나 가볍게 몸을 풀어주는 것만으로도 통증을 줄일 수는 있지만, 너무 오래 방치할 경우에는 부


In [44]:
# text를 받아 생성한 문장을 반환하는 함수
def generate_text(text):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    gen_ids = model.generate(input_ids,
                            max_length=512,
                            repetition_penalty=5.0,
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    return generated

In [45]:
text = '봄 여행지 추천?'
generate_text(text)

'봄 여행지 추천?</d>'

In [46]:
text = '오늘의 점심 식사는?'
generate_text(text)

'오늘의 점심 식사는?"\n"아니야. 그게 아니라면."\n그녀는 고개를 끄덕였다.\n그리고는 다시 한 번 말했다.\n\'그래, 내가 너를 사랑하고 있다는 걸 알았어!\'\n이제 그녀는 자신의 마음을 다잡았다.\n하지만 그녀의 마음은 여전히 불안했다.\n왜냐하면 그녀가 자신을 사랑하는 것은 그녀 자신이기 때문이다.\n그러나 그녀를 향한 그의 사랑은 결코 끝나지 않았다.\n그는 그녀에게 사랑을 고백하지 않을 수 없었다.\n그러자 그는 더 이상 그를 미워할 수가 없게 되었다.\n결국 그가 그에게로 돌아가지 않기로 결심했다.\n그의 마음속에 있는 모든 것을 빼앗아버리고 싶었다.\n더 이상은 견딜 수도 없어.\n나는 이제 그만 돌아가고 싶어!\n내가 원하는 건 오직 하나,\n나의 사랑이 아니란 말이지?\n내 안에 있던 것들을 모두 빼앗아 버릴 거야, 라고 생각했다.\n지금까지 해왔던 것처럼 말이다.\n다시 말해서, 나는 지금 이 순간부터 내 안의 어떤 것까지도 전부 잃어버리게 될 것이다.\n모든 것이 사라져 버린다는 사실을 깨달을 때까지만이라도 당신을 용서해 주겠다.\n당신이 나를 위해 무엇을 할 것인가?\n나를 위해서 무엇이든 해야 한다.\n나에게 필요한 것, 그리고 나 자신에게 필요했던 것들이 무엇인지 알고 있다.\n또한 당신의 삶을 바꿀 수는 없다.\n다음 날 아침은 너무나도 행복하게 느껴졌다.\n마지막으로 우리는 서로를 향해 손을 흔들며 인사를 나누었다.\n우리는 서로의 얼굴을 마주보며 이야기를 나누었고, 서로에 대한 그리움을 달랬다.</d>'

In [47]:
text = '샐러드 레시피'
generate_text(text)

'샐러드 레시피를 공개했다.\n이날 방송에서 김 대표는 “요즘은 다이어트 식품으로 많이 찾는 것이 사실”이라며 “이런 식품을 먹으면서 칼로리가 높은 음식을 먹는 것은 좋지 않다”고 말했다.\n이어 그는 “아무리 좋은 음식이라도 몸에 나쁜 영향을 끼치면 안 된다”면서 “그렇기 때문에 평소에는 잘 먹지 않는다. 특히 과식을 하지 않도록 주의해야 한다”고 덧붙였다.\n김 대표가 소개한 ‘다크초콜릿’은 초콜릿의 단맛을 내는 폴리페놀 성분이 들어있어 체내 흡수율을 높여준다.\n또한 비타민C가 풍부해 피로회복에 도움을 준다.\n특히, 다크로쉐에 함유된 안토시아닌 성분은 혈중 콜레스테롤 수치를 낮춰주는 효과가 있다.\n아울러 항산화 물질인 베타카로틴과 칼슘도 풍부하다.\n이에 따라 당뇨병 환자나 고혈압 환자 등 만성질환을 가진 사람은 물론 비만이나 고지혈증 환자에게 좋다.\n최근엔 식이섬유와 섬유질이 풍부한 건강기능식품들이 인기를 끌고 있다.</d>'

In [48]:
generate_text("파이썬 코딩 공부 방법 추천")

"파이썬 코딩 공부 방법 추천\n1. 딥러닝을 활용한 학습법 소개를 통해 자신의 실력을 점검해보자.\n2. '스마트폰으로 촬영한 이미지'를 활용해 자신만의 이미지를 만들어보자.</d>"